In [1]:
import sys
import string
import os
import numpy as np
from math import pi
import scipy.stats
import warnings
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

from bokeh.layouts import gridplot
from bokeh.plotting import output_file, figure, show
from bokeh.palettes import inferno
from bokeh.models import Label
from bokeh.models import Range1d
from bokeh.io import export_png
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
fig = plt.figure()
mpl.rcParams['figure.figsize'] = (10.0, 6.0)

<Figure size 432x288 with 0 Axes>

In [3]:
print('Write the length of DNA in bp:')
bp = int(input().strip())
print('Write the concentration of NaCl in system:')
mM = int(input().strip())
step = 1
fname = f'../DNA/dsDNA/{bp}bp/{mM}mM/push_from_30nm/dsDNA_{bp}bp_{mM}mM_pressure_push.dat'

rads = np.array([float(i.split('\t')[1].strip()) for i in open(fname)])
frames = np.array([int(i.split('\t')[0].strip()) for i in open(fname)]) // 10000

Write the length of DNA in bp:



ValueError: invalid literal for int() with base 10: ''

In [ ]:
#Check radial distribution for trajectories
bp = 4000
mM = 50
if not os.path.exists(f'../DNA/dsDNA/plots'):
    os.mkdir(f'../DNA/dsDNA/plots')
if not os.path.exists(f'../DNA/dsDNA/plots/rad_distr_traj'):
    os.mkdir(f'../DNA/dsDNA/plots/rad_distr_traj')
# elems = ['DNA']
runs = ['1','2','3','4','5']

for frame in tqdm(range(0,9030,10)):
    #Параметры для графика BOKEH
    p = figure(x_axis_label='r, nm', y_axis_label=r'g(r)', 
            plot_width=800, plot_height=600,
           x_range = (0, 61), y_range = (0, 0.022))
    p.xaxis.axis_label_text_font_size = "22pt"
    p.xaxis.major_label_text_font_size = "18pt"
    p.yaxis.axis_label_text_font_size = "22pt"
    p.yaxis.major_label_text_font_size = "18pt"
    p.toolbar.logo = None
    p.toolbar_location = None
    colors = inferno(len(runs)+1)
    for c, run in zip(colors,runs):
        x = np.arange(1,600)
        fname = f'../DNA/dsDNA/{bp}bp/{mM}mM/push_100k_steps/rad_distr/run_{run}/DNA/{frame}.dat'
        dens = np.array([float(i.split('\t')[1].strip()) for i in open(fname)])
#         print(len(dens))
        p.line(x/10, dens, color = c, legend=run, line_width = 5)
    p.legend.label_text_font_size = "18pt"
    export_png(p, filename=f'../DNA/dsDNA/plots/rad_distr_traj/{frame//10}.png')
    

In [5]:
#Check radial distribution for different mMs
if not os.path.exists(f'../DNA/dsDNA/plots/rad_distr_mMs'):
    os.mkdir(f'../DNA/dsDNA/plots/rad_distr_mMs')
elems = ['DNA','Na','Cl']
bp = 4000
mMs = ['0.1','2','5','12','50']
colors = inferno(len(elems)+1)
boxes = [1000,1000,1250,950]

for mM, x_max in zip(mMs,boxes):
    x = np.arange(1,x_max)
    print(mM, x_max)
    fname = f'../DNA/dsDNA/{bp}bp/{mM}mM/push_from_30nm/dsDNA_{bp}bp_{mM}mM_pressure_push.dat'
    frames = np.array([int(i.split('\t')[0].strip()) for i in open(fname)]) // 10000
    rads = np.array([float(i.split('\t')[1].strip()) for i in open(fname)])
#     for r in np.linspace(29.9,11.0,5):
    inds = np.where(rads==11.030499)[0]
    frame = frames[inds][0]
    p = figure(x_axis_label='r, nm', y_axis_label=r'g(r)', 
            plot_width=800, plot_height=600,
            y_range = (0, 0.022))
    p.xaxis.axis_label_text_font_size = "22pt"
    p.xaxis.major_label_text_font_size = "18pt"
    p.yaxis.axis_label_text_font_size = "22pt"
    p.yaxis.major_label_text_font_size = "18pt"
    p.toolbar.logo = None
    p.toolbar_location = None

    for c, elem in zip(colors, elems):
        fname = f'../DNA/dsDNA/{bp}bp/{mM}mM/push_from_30nm/rad_distr/{elem}/{frame}.dat'
        dens = np.array([float(i.split('\t')[1].strip()) for i in open(fname)])
        p.line(x/10, dens, color = c, legend=elem, line_width = 3)
    p.legend.label_text_font_size = "18pt"
    show(p)
#     export_png(p, filename=f'../DNA/dsDNA/plots/rad_distr_mMs/mM{mM}_r{rads[inds][0]}.png')


0.1 1000
2 1000
5 1250
12 950


In [65]:
#Check radial distribution for different mMs
if not os.path.exists(f'../DNA/dsDNA/plots/rad_distr_elems_mMs'):
    os.mkdir(f'../DNA/dsDNA/plots/rad_distr_elems_mMs')
elems = ['Cl']
bp = 4000
mMs = ['0.1','2','5','12']
colors = inferno(len(mMs)+1)
boxes = [1000,1000,1250,950]

for elem in elems:
    p = figure(x_axis_label='r, nm', y_axis_label=r'g(r) · e-3', 
            plot_width=800, plot_height=600,
            x_range = (0,31), y_range = (0,1.44))
    p.xaxis.axis_label_text_font_size = "22pt"
    p.xaxis.major_label_text_font_size = "18pt"
    p.yaxis.axis_label_text_font_size = "22pt"
    p.yaxis.major_label_text_font_size = "18pt"
    p.toolbar.logo = None
    p.toolbar_location = None

    for c, mM, x_max in zip(colors, mMs, boxes):
        x = np.arange(1,x_max)
        print(mM, x_max)
        fname = f'../DNA/dsDNA/{bp}bp/{mM}mM/push_from_30nm/dsDNA_{bp}bp_{mM}mM_pressure_push.dat'
        frames = np.array([int(i.split('\t')[0].strip()) for i in open(fname)]) // 10000
        rads = np.array([float(i.split('\t')[1].strip()) for i in open(fname)])
    #     for r in np.linspace(29.9,11.0,5):
        inds = np.where(rads==11.030499)[0]
        frame = frames[inds][0]
        fname = f'../DNA/dsDNA/{bp}bp/{mM}mM/push_from_30nm/rad_distr/{elem}/{frame}.dat'
        dens = np.array([float(i.split('\t')[1].strip()) for i in open(fname)])
        p.line(x/10, dens*1e3, color = c, legend=mM, line_width = 3)
    p.legend.label_text_font_size = "18pt"
    p.legend.location = "top_right"
#     show(p)
    export_png(p, filename=f'../DNA/dsDNA/plots/rad_distr_elems_mMs/{elem}_r{rads[inds][0]}.png')


0.1 1000
2 1000
5 1250
12 950


In [4]:
#Check radial distribution for trajectories
bp = 4000
mM = 2
# if not os.path.exists(f'../DNA/dsDNA/plots'):
#     os.mkdir(f'../DNA/dsDNA/plots')
# if not os.path.exists(f'../DNA/dsDNA/plots/rad_distr_traj'):
#     os.mkdir(f'../DNA/dsDNA/plots/rad_distr_traj')
elems = ['DNA','Na','Cl']


for frame in tqdm(range(3589,3590)):
    #Параметры для графика BOKEH
    p = figure(x_axis_label='r, nm', y_axis_label=r'g(r)', 
            plot_width=800, plot_height=600,
            x_range = (0,31))#, y_range = (0, 0.001))
    p.xaxis.axis_label_text_font_size = "22pt"
    p.xaxis.major_label_text_font_size = "18pt"
    p.yaxis.axis_label_text_font_size = "22pt"
    p.yaxis.major_label_text_font_size = "18pt"
    p.toolbar.logo = None
    p.toolbar_location = None
    colors = inferno(2*len(elems)+1)
    for c, elem in zip(colors,elems):
        x1 = np.arange(1,2000)
        fname1 = f'../DNA/dsDNA/{bp}bp/{mM}mM/push_from_30nm/rad_distr_check/Cl/{frame}.dat'
        dens1 = np.array([float(i.split('\t')[1].strip()) for i in open(fname1)])
        x2 = np.arange(1,300)
        fname2 = f'../DNA/dsDNA/{bp}bp/{mM}mM/push_from_30nm/rad_distr/Cl/{frame}.dat'
        dens2 = np.array([float(i.split('\t')[1].strip()) for i in open(fname2)])
#         print(len(dens))
        p.line(x1/10, dens1, color = colors[1], legend='check', line_width = 5)
        p.line(x2/10, dens2, color = colors[3], legend='true', line_width = 2)
    p.legend.label_text_font_size = "18pt"
#     export_png(p, filename=f'../DNA/dsDNA/plots/rad_distr_traj/{frame//10}.png')
    show(p)

  0%|          | 0/1 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../DNA/dsDNA/4000bp/2mM/push_from_30nm/rad_distr_check/Cl/3589.dat'